In [1]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
# from IPython.display import clear_output
import os
import random
from tqdm.auto import tqdm
from timeit import default_timer as timer

import tensorflow as tf
import keras
import keras.backend as K
import keras.layers as KL
# import keras.engine as KE
import keras.models as KM
from tensorflow.keras.initializers import GlorotNormal

tf.get_logger().setLevel('ERROR')

print("TensorFlow version:", tf.__version__)

Mounted at /content/drive
TensorFlow version: 2.13.0


In [2]:
!pip install wandb
import wandb
# !wandb login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 24.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=d3a3e1f376c2969296e8fe5d5dee41c99edf0264b72e8458c915416bd0c18196
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [3]:
import sys
sys.path.append('/content/drive/My Drive/kaggle/contrail_detector/')
import utils
import models
from configs import unet_baseline_config, wnet_unet_config, wnet_rcnn_config, alt_wnet_unet_config, alt_wnet_rcnn_config, Path

In [4]:
# this model class only havs inference mode

class wnet(keras.Model) :
  def __init__(self, unet, rcnn = None, binary_classifier = None) :
    super(wnet, self).__init__()
    self.binary_classifier = binary_classifier
    self.rcnn = rcnn
    self.unet = unet

  def call(self, images, normalized_anchors, gt_bboxes) :
    self.rcnn.mode = "inference"
    masks = self.unet(images)[...,-1:]

    if self.rcnn :
      attention_masks = self.rcnn(images, normalized_anchors, gt_bboxes = gt_bboxes)["attention_masks"]
      attention_masks = tf.expand_dims(attention_masks, axis = -1)
    else :
      attention_masks = tf.ones((images.shape[0], 1, 1, 1))

    if self.binary_classifier :
      contrail_exists = self.binary_classifier(images)[:, tf.newaxis, tf.newaxis, tf.newaxis, tf.newaxis]
    else :
      contrail_exists = tf.ones((images.shape[0], 1, 1, 1))

    return masks, attention_masks, contrail_exists


def dice_score(y_true, y_pred, thr=0.5, epsilon=1e-6):
    # Thresholding predictions
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    y_pred = tf.cast(y_pred > thr, tf.float32)

    # Flattening tensors
    y_true = tf.reshape(y_true, [tf.shape(y_true)[0], -1])
    y_pred = tf.reshape(y_pred, [tf.shape(y_pred)[0], -1])

    # Computing intersections and unions
    intersections = tf.reduce_sum(y_true * y_pred, axis=1)
    unions = tf.reduce_sum(y_true, axis=1) + tf.reduce_sum(y_pred, axis=1)

    # Computing Dice scores
    dices = (2. * intersections + epsilon) / (unions + epsilon)

    return tf.reduce_mean(dices)

# 0. Load Test Dataset and Models




In [5]:
#make a dataset object using test data
test_metadata = pd.read_json("/content/drive/MyDrive/kaggle/contrail_detector/data/test_metadata.json", dtype={'record_id': 'str'})

def load_data(record_id, path):
    # make path objects
    record_id = record_id.numpy().decode('utf-8')
    path = path.numpy().decode('utf-8')
    image_path = path + "image/" + str(record_id) + ".npy"
    mask_path = path + "mask/" + str(record_id) + ".npy"
    gt_boxes_path = path + "gt_bboxes/" + str(record_id) + ".npy"
    # load the images and masks for the correspoding paths
    normalized_anchors = np.load("/content/drive/MyDrive/kaggle/contrail_detector/data/normalized_anchors.npy").astype(np.float32)
    image = np.load(image_path)[...,wnet_unet_config.n_times_before, :].astype(np.float32)
    mask = np.load(mask_path).astype(np.float32)
    gt_boxes = np.load(gt_boxes_path).astype(np.float32)

    return image, normalized_anchors, mask, gt_boxes


id_to_data = lambda record_id, path : tf.py_function(func = load_data, inp = [record_id, path], Tout = (tf.float32, tf.float32, tf.float32, tf.float32))
test_id = test_metadata.record_id.to_list()

test_path = len(test_id)*[Path.test]


test_id_dataset = tf.data.Dataset.from_tensor_slices(test_id)
test_path_dataset = tf.data.Dataset.from_tensor_slices(test_path)


test_dataset = tf.data.Dataset.zip((test_id_dataset, test_path_dataset)).map(id_to_data).shuffle(wnet_unet_config.buffer_size).batch(wnet_unet_config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
models_dict = {"binary_classifier" : None,
               "unet_baseline" : None,
               "wnet_unet" : None,
               "alt_wnet_unet" : None,
               "wnet_rcnn" : None,
               "alt_wnet_rcnn" : None}

In [7]:
# recover binary_classifer from W&B

binary_classifer = None

models_dict["binary_classifier"] = binary_classifer

In [8]:
# Initialize wandb once
run = wandb.init()

def recover_unet(artifact_name, config, epoch):
  # Use the artifact
  artifact = run.use_artifact(artifact_name, type='model')
  # Download the artifact to the specified directory
  download_path = artifact.download()
  # Load the weights into the model
  model = models.unet()
  lr_schedule = tf.keras.optimizers.schedules.CosineDecay(config.initial_learning_rate,
                                                          decay_steps=config.decay_steps,
                                                          alpha=config.alpha)
  optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
  checkpoint = tf.train.Checkpoint(step=tf.Variable(0),
                                    optimizer=optimizer,
                                    model=model)
  checkpoint.restore(f'{download_path}/ckpt-{epoch}')
  return model

# Recover unet models
models_dict["unet_baseline"] = recover_unet('jun23ird/contrail_detector/unet_baseline-epoch-34-checkpoint:v0', unet_baseline_config, 34)
models_dict["wnet_unet"] = recover_unet('jun23ird/contrail_detector/wnet_unet-epoch-32-checkpoint:v0',  wnet_unet_config, 32)
models_dict["alt_wnet_unet"] = recover_unet('jun23ird/contrail_detector/alt_wnet_unet-epoch-54-checkpoint:v0',  alt_wnet_unet_config, 54)


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Downloading large artifact unet_baseline-epoch-34-checkpoint:v0, 431.04MB. 2 files... 
wandb:   2 of 2 files downloaded.  
Done. 0:0:13.7
wandb: Downloading large artifact wnet_unet-epoch-32-checkpoint:v0, 431.04MB. 2 files... 
wandb:   2 of 2 files downloaded.  
Done. 0:0:6.8
wandb: Downloading large artifact alt_wnet_unet-epoch-54-checkpoint:v0, 431.04MB. 2 files... 
wandb:   2 of 2 files downloaded.  
Done. 0:0:7.7


In [9]:
def recover_rcnn(artifact_name, config, epoch):
  # Use the artifact
  artifact = run.use_artifact(artifact_name, type='model')
  # Download the artifact to the specified directory
  download_path = artifact.download()
  # Load the weights into the model
  model = models.rcnn('training', config, architecture="resnet50")
  lr_schedule = tf.keras.optimizers.schedules.CosineDecay(config.initial_learning_rate,
                                                          decay_steps=config.decay_steps,
                                                          alpha=config.alpha)
  optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule,
                                       clipnorm = config.gradient_clipnorm)
  checkpoint = tf.train.Checkpoint(step=tf.Variable(0),
                                    optimizer=optimizer,
                                    model=model)
  checkpoint.restore(f'{download_path}/ckpt-{epoch}')
  return model

# recover models
models_dict["wnet_rcnn"] = recover_rcnn('jun23ird/contrail_detector/wnet_rcnn-epoch-42-checkpoint:v0',  wnet_rcnn_config, 84)
models_dict["alt_wnet_rcnn"] = recover_rcnn('jun23ird/contrail_detector/alt_wnet_rcnn-epoch-7-checkpoint:v1',  alt_wnet_rcnn_config, 14)

wandb: Downloading large artifact wnet_rcnn-epoch-42-checkpoint:v0, 1101.79MB. 2 files... 
wandb:   2 of 2 files downloaded.  
Done. 0:0:29.5


102967424/102967424 [==============================] - 1s 0us/step


wandb: Downloading large artifact alt_wnet_rcnn-epoch-7-checkpoint:v1, 1101.79MB. 2 files... 
wandb:   2 of 2 files downloaded.  
Done. 0:0:16.9


# 1. Test Models

In [10]:
def test_model(model, dataset = test_dataset) :
  # make predictions using wnet against training data
  original_images = []
  ground_truths = []
  predictions = []

  for step, inputs_batch in enumerate(tqdm(dataset, desc='Validation', position=0)):
    image_batch, normalized_anchors_batch, mask_batch, gt_bboxes = inputs_batch
    original_images += [image_batch]
    ground_truths += [mask_batch]
    masks, attention_masks, contrail_exists = model(image_batch, normalized_anchors_batch, gt_bboxes = gt_bboxes)
    # Casting tensors to float32 before multiplication
    masks = tf.cast(masks, tf.float32)
    attention_masks = tf.cast(attention_masks, tf.float32)
    contrail_exists = tf.cast(contrail_exists, tf.float32)
    predictions += [masks*attention_masks*contrail_exists]


  original_images = tf.concat(original_images, axis = 0)
  ground_truths = tf.concat(ground_truths, axis = 0)
  predictions = tf.concat(predictions, axis = 0)

  return original_images, ground_truths, predictions

-  U-Net = unet_baseline, R-CNN = wnet_rcnn

In [ ]:
# initialize wnet object
model = models.wnet(models_dict["unet_baseline"],
                    models_dict["wnet_rcnn"])
# test the wnet model against test dataset
original_images, ground_truths, predictions = test_model(model)

# show best and worst predictions
best_threshold = 1.0

# compute the dice score
print(f"dice score : {dice_score(ground_truths, predictions, best_threshold).numpy()}")
# show predictions
utils.show_predictions(original_images, ground_truths, predictions, best_threshold, 0.001, 50)

- U-Net = unet_baseline, R-CNN = alt_wnet_rcnn

In [ ]:
# initialize wnet object
model = models.wnet(models_dict["unet_baseline"],
                    models_dict["alt_wnet_rcnn"])
# test the wnet model against test dataset
original_images, ground_truths, predictions = test_model(model)

# show best and worst predictions
best_threshold = 1.0

# compute the dice score
print(f"dice score : {dice_score(ground_truths, predictions, best_threshold).numpy()}")
# show predictions
utils.show_predictions(original_images, ground_truths, predictions, best_threshold, 0.001, 50)

- U-Net = wnet_unet, R-CNN = wnet_rcnn

In [ ]:
# initialize wnet object
model = models.wnet(models_dict["wnet_unet"],
                    models_dict["wnet_rcnn"])
# test the wnet model against test dataset
original_images, ground_truths, predictions = test_model(model)

# show best and worst predictions
best_threshold = 0.99

# compute the dice score
print(f"dice score : {dice_score(ground_truths, predictions, best_threshold).numpy()}")
# show predictions
utils.show_predictions(original_images, ground_truths, predictions, best_threshold, 0.001, 50)

- U-Net = wnet_unet, R-CNN = alt_wnet_rcnn

In [ ]:
# initialize wnet object
model = models.wnet(models_dict["wnet_unet"],
                    models_dict["alt_wnet_rcnn"])
# test the wnet model against test dataset
original_images, ground_truths, predictions = test_model(model)

# show best and worst predictions
best_threshold = 0.99

# compute the dice score
print(f"dice score : {dice_score(ground_truths, predictions, best_threshold).numpy()}")
# show predictions
utils.show_predictions(original_images, ground_truths, predictions, best_threshold, 0.001, 50)

- U-Net = alt_wnet_unet, R-CNN = wnet_rcnn

In [ ]:
# initialize wnet object
model = models.wnet(models_dict["alt_wnet_unet"],
                    models_dict["wnet_rcnn"])
# test the wnet model against test dataset
original_images, ground_truths, predictions = test_model(model)

# show best and worst predictions
best_threshold = 1.0

# compute the dice score
print(f"dice score : {dice_score(ground_truths, predictions, best_threshold).numpy()}")
# show predictions
utils.show_predictions(original_images, ground_truths, predictions, best_threshold, 0.001, 50)

- U-Net = alt_wnet_unet, R-CNN = alt_wnet_rcnn

In [ ]:
# initialize wnet object
model = models.wnet(models_dict["alt_wnet_unet"],
                    models_dict["alt_wnet_rcnn"])
# test the wnet model against test dataset
original_images, ground_truths, predictions = test_model(model)

# show best and worst predictions
best_threshold = 1.0

# compute the dice score
print(f"dice score : {dice_score(ground_truths, predictions, best_threshold).numpy()}")
# show predictions
utils.show_predictions(original_images, ground_truths, predictions, best_threshold, 0.001, 50)